In [1]:
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [2]:
# Download required data from nltk.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shefi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shefi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        # add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower())
         for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(
        word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Heylo'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Cya'], 'goodbye'), (['Bubye'], 'goodbye'), (['What', 'is', 'a', 'coronavirus'], 'What is a coronavirus?'), (['What', 'is', 'covid', '19'], 'What is COVID-19?'), (['symptoms', 'of', 'covid', '19'], 'What are the symptoms of COVID 19?'), (['symptoms', 'of', 'coronavirus'], 'What are the symptoms of COVID 19?'), (['covid', '19', 'symptoms'], 'What are the symptoms of COVID 19?'), (['coronavirus', 'symptoms'], 'What are the symptoms of COVID 19?'), (['How', 'does', 'COVID', '19', 'spread'], 'How does COVID 19 spread?'), (['How', 'does', 'coronavirus', 'spread'], 'How does COVID 19 spread?'), (['Wh

C:\Users\shefi\AppData\Local\Temp/ipykernel_4508/1830752982.py:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200


C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


14/14 [==============================] - 0s 1ms/step - loss: 3.2998 - accuracy: 0.0294
Epoch 2/200
14/14 [==============================] - 0s 2ms/step - loss: 3.1354 - accuracy: 0.0588
Epoch 3/200
14/14 [==============================] - 0s 1ms/step - loss: 3.0160 - accuracy: 0.1765
Epoch 4/200
14/14 [==============================] - 0s 1ms/step - loss: 2.8956 - accuracy: 0.2353
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 2.7546 - accuracy: 0.2500
Epoch 6/200
14/14 [==============================] - 0s 1ms/step - loss: 2.6083 - accuracy: 0.3088
Epoch 7/200
14/14 [==============================] - 0s 2ms/step - loss: 2.4570 - accuracy: 0.3971
Epoch 8/200
14/14 [==============================] - 0s 2ms/step - loss: 2.2136 - accuracy: 0.5441
Epoch 9/200
14/14 [==============================] - 0s 2ms/step - loss: 2.3308 - accuracy: 0.3676
Epoch 10/200
14/14 [==============================] - 0s 1ms/step - loss: 2.0565 - accuracy: 0.4412
Epoch 11/200
14/14 [=

14/14 [==============================] - 0s 1ms/step - loss: 0.1224 - accuracy: 1.0000
Epoch 84/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1243 - accuracy: 0.9559
Epoch 85/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1371 - accuracy: 0.9853
Epoch 86/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1322 - accuracy: 0.9559
Epoch 87/200
14/14 [==============================] - 0s 997us/step - loss: 0.1120 - accuracy: 0.9706
Epoch 88/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1160 - accuracy: 0.9853
Epoch 89/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1487 - accuracy: 0.9559
Epoch 90/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0910 - accuracy: 0.9706
Epoch 91/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1821 - accuracy: 0.9118
Epoch 92/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0946 - accuracy: 0.9706
Epoch 93/20

14/14 [==============================] - 0s 1ms/step - loss: 0.1352 - accuracy: 0.9706
Epoch 165/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0857 - accuracy: 0.9853
Epoch 166/200
14/14 [==============================] - 0s 1ms/step - loss: 0.2318 - accuracy: 0.9559
Epoch 167/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0393 - accuracy: 0.9853
Epoch 168/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0620 - accuracy: 0.9853
Epoch 169/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1364 - accuracy: 0.9559
Epoch 170/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1160 - accuracy: 0.9559
Epoch 171/200
14/14 [==============================] - 0s 997us/step - loss: 0.0480 - accuracy: 0.9853
Epoch 172/200
14/14 [==============================] - 0s 914us/step - loss: 0.0962 - accuracy: 0.9706
Epoch 173/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0719 - accuracy: 0.9853


In [5]:
# pip install tensorflow==2.8.0
# pip install Keras

In [6]:
import os
import random
import json
from keras.models import load_model
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [7]:
pwd

'C:\\Users\\shefi\\Downloads\\covid_chatbot\\chatbot\\components'

In [8]:
nltk.download('omw-1.4')
model = load_model(os.path.join( 'chatbot_model.h5'))
intents = json.loads(
    open(os.path.join( 'intents.json')).read())
# intents = json.loads(open(os.path.join( 'intents.json')).read().decode("utf-8-sig"))
words = pickle.load(
    open(os.path.join(  'words.pkl'), 'rb'))
classes = pickle.load(
    open(os.path.join(  'classes.pkl'), 'rb'))

print('words', words)
print('classes', classes)


def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                #if show_details:
                    #print("found in bag: %s" % word)
    #print("np.array(bag)", np.array(bag))
    return(np.array(bag))


def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words, show_details=True)
    flag = False
    return_list = []
    for x in p:
        if x == 1:
            flag = True
    if (flag == True):
        res = model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
        # sorting strength probability
        results.sort(key=lambda x: x[1], reverse=True)

        #print("result", results)
        for r in results:
            return_list.append(
                {"intent": classes[r[0]], "probability": str(r[1])})
    else:
        return_list.append({"intent": "noanswer", "probability": str(1)})

    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg)
    #print('ints', ints)
    res = getResponse(ints, intents)
    #print('res', res)
    return res

words ["'", "'booster", "'s", "'three", '(', ')', '19', 'a', 'about', 'after', 'alot', 'am', 'an', 'and', 'antibiotic', 'any', 'anyone', 'appointment', 'are', 'awesome', 'between', 'book', 'booster', 'bubye', 'by', 'bye', 'can', 'card', 'catch', 'cause', 'chatting', 'clinic', 'coronavirus', 'covid', 'covid-19', 'cure', 'cya', 'difference', 'different', 'disease', 'do', 'doe', 'dose', 'drug', 'effect', 'effected', 'effective', 'even', 'everyone', 'flu', 'follow', 'for', 'from', 'get', 'goodbye', 'have', 'having', 'health', 'hello', 'helpful', 'helping', 'hey', 'heylo', 'hi', 'hola', 'how', 'i', 'if', 'in', 'incubation', 'infection', 'influenza', 'is', 'last', 'later', 'likely', 'long', 'major', 'me', 'measure', 'medicine', 'much', 'myself', 'next', 'nice', 'not', 'of', 'ontario', 'or', 'para', 'period', 'positive', 'prevent', 'preventing', 'primary', 'protect', 'protection', 'public', 'red', 'required', 'see', 'series', 'should', 'side', 'so', 'spread', 'still', 'surface', 'symptom', 't

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\shefi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# test chatbot 
from termcolor import colored
print ("CHAT START (BLANK AND ENTER TO STOP)")
try:
    while (1):
        x=input(colored("YOU:",'red'))
        if (x.strip()==""):
            break
        print ( colored("CHATBOT:",'blue'), colored(chatbot_response(x),'green'))
except:
    print("ERROR")

print("CHAT END")


CHAT START (BLANK AND ENTER TO STOP)
YOU:how can i prevent from coronavirus
CHATBOT: The best way to prevent COVID-19 is to get vaccinated with an FDA-approved or FDA-authorized COVID-19 vaccine and stay up to date on your COVID-19 vaccines. In addition, the CDC recommends everyday preventive actions to help prevent the spread of COVID-19.
